# 구조적 API 기본 연산
- DataFrame의 구성
  - Row 타입의 레코드
  - 레코드에 수행할 연산 표현식을 나타내는 여러 컬럼
- 스키마: 각 컬럼명과 데이터 타입을 정의
- 파티셔닝: DataFrame이나 Dataset이 클러스터에서 물리적으로 배치되는 형태를 정의
- 파티셔닝 스키마: 파티션을 배치하는 방법을 정의

In [0]:
path="/FileStore/tables/2015_summary.json"

In [0]:
#DataFrame 생성
df= spark.read.format('json').load(path)

In [0]:
#스키마 출력
df.printSchema()

root
-- DEST_COUNTRY_NAME: string (nullable = true)
-- ORIGIN_COUNTRY_NAME: string (nullable = true)
-- count: long (nullable = true)

## 스키마
- <strong>데이터소스에서 스키마를 얻거나 직접 정의 가능</strong>
- 데이터를 읽기 전에 스키마를 정의해야 하는지 여부는 상황에 따라 달라짐
  - ETL 작업에 스파크를 사용한다면 직접 스키마를 정의해야함
  - ETL 작업 중 데이터 타입을 알기 힘든 CSV나 JSON 등의 데이터소스를 사용하는 경우, 스키마 추론 과정에서 읽어 들인 샘플로 스키마를 결정해버릴 수 있음

In [0]:
spark.read.format('json').load(path).schema

Out[3]: StructType(List(StructField(DEST_COUNTRY_NAME,StringType,true),StructField(ORIGIN_COUNTRY_NAME,StringType,true),StructField(count,LongType,true)))

------
- 스키마는 여러 개의 StructField 타입 필드로 구성된 StructType 객체임

In [0]:
from pyspark.sql import types as T
#직접 스키마 정의
myManualSchema = T.StructType([
  T.StructField('DEST_COUNTRY_NAME', T.StringType(), True),
  T.StructField('ORIGIN_COUNTRY_NAME', T.StringType(), True),
  T.StructField('count', T.LongType(), False, metadata={'hello':'world'})
])

In [0]:
#스키마 적용
df2 = spark.read.format('json').schema(myManualSchema).load(path)

## 컬럼과 표현식
- 스파크의 컬럼은 <strong>표현식을 사용해 레코드 단위로 계산한 값을 단순하게 나타내는 논리적인 구조</strong>
  - 표현식으로 컬럼을 선택, 조작, 제거 가능
- 따라서 컬럼의 실젯값을 얻으려면 로우가 필요하고, 
- 로우를 얻으려면 DataFrame이 필요하다.
- DataFrame을 통하지 않으면 <strong>외부에서 컬럼에 접근할 수 없다.</strong>
- 컬럼 내용을 수정하려면 반드시 DataFrame의 스파크 트랜스포메이션을 사용해야함

### 컬럼
- col함수나 column함수를 사용하여 컬럼을 생성하고 참조

In [0]:
from pyspark.sql import functions as F
#col함수로 컬럼 생성
F.col('someColumnName')

Out[11]: Column<'someColumnName'>

------
- 컬럼이 DataFrame에 있을지 없을지는 알 수 없다.
- 컬럼은 컬럼명을 카탈로그에 저장된 정보와 비교하기 전까지 <strong>미확인</strong> 상태
- 4장에서 본 것 처럼, <strong>분석기가 동작하는 단계에서 컬럼과 테이블을 분석함</strong>

#### 명시적 컬럼 참조
- DataFrame의 컬럼은 <strong>col 메서드</strong>로 참조함
- col메서드를 사용해 <strong>명시적으로 컬럼을 정의</strong>하면 스파크는 분석기 실행 단계에서 <strong>컬럼 확인 절차를 생략함</strong>

### 표현식
- 앞서 컬럼은 표현식이라고 했음 -> 표현식이란?
 - DataFrame 레코드의 여러 값에 대한 트랜스포메이션 집합